In [1]:
import numpy as np

## Benchmark Multimodal Functions Available
| Function | Dimension | Bounds | Optimal Function Value |
| -------- | --------- | ------ | ---------------------- |
| $$ f_{1} = 4x_1^2 - 2.1x_1^4 + \frac{1}{3}x_1^6 + x_1 x_2 - 4x_2^2 + 4 x_2^4 $$ | 2 | [-5, 5] | -1.0316 |
| $$ f_{2} = (x_2 - \frac{5.1}{4\pi^2}x_1^2 + \frac{5}{\pi}x_1 -6)^2 +10(1 - \frac{1}{8\pi})\cos{x_1} + 10 $$ | 2 | [-5, 5] | 0.398 |
| $$ f_{3} = -\sum_{i=1}^{4} c_i exp(-\sum_{j=1}^{3} a_{ij}(x_j - p_{ij})^2) $$ | 3 | [1, 3] | -3.86 |
| $$ f_{4} = -\sum_{i=1}^{4} c_i exp(-\sum_{j=1}^{6} a_{ij}(x_j - p_{ij})^2) $$ | 6 | [0, 1] | -3.32 |
| $$ f_{5} = -\sum_{i=1}^{7} [(X - a_i)(X - a_i)^T + c_i]^{-1} $$ | 4 | [0, 10] | -10.4028 |

In [62]:
class Function:
    def __init__(self, x = None, n = 2, lb = np.array([-5, -5]), ub = np.array([5, 5])):
        self.n_x = n
        if x is not None:
            assert(x.shape[0] == self.n_x)
            self.fvalue = self.getFValue(x)
        self.x = x
        self.fvalue = None
        assert(lb.shape[0] == self.n_x)
        self.lb = lb
        assert(ub.shape[0] == self.n_x)
        self.ub = ub
        self.benchmark_selected = None
    
    def setBenchmarkFunction(self, f_name = "f1"):
        benchmarks = {
            "f1": [2, np.array([-5, -5]), np.array([5, 5])],
            "f2": [2, np.array([-5, -5]), np.array([5, 5])],
            "f3": [3, 1*np.ones(shape=(3,)), 3*np.ones(shape=(3,))],
            "f4": [6, np.zeros(shape=(6,)), 1*np.ones(shape=(6,))],
            "f5": [4, np.zeros(shape=(4,)), 10*np.ones(shape=(4,))]
        }
        self.benchmark_selected = f_name
        [self.n_x, self.lb, self.ub] = benchmarks.get(f_name, benchmarks.get("f1"))
    
    def isFeasible(self, x):
        return np.all(x >= self.lb) and np.all(x <= self.ub)
    
    def getFValue(self, x):
        if self.benchmark_selected is None:
            func_value = 4*x[0]**2 - 2.1*x[0]**4 + (x[0]**6)/3 + x[0]*x[1] - 4*x[1]**2 + 4*x[1]**4
            return func_value
        benchmarks_coeffs = {
            "f3": {"a": np.array([[3, 10, 30], [0.1, 10, 35], [3, 10, 30], [0.1, 10, 35]]),
                   "c": np.array([1, 1.2, 3, 3.2]),
                   "p": np.array([[0.3689, 0.117, 0.2673], [0.4699, 0.4387, 0.747], [0.1091, 0.8732, 0.5547], [0.03815, 0.5743, 0.8828]])},
            "f4": {"a": np.array([[10, 3, 17, 3.5, 1.7, 8], [0.05, 10, 17, 0.1, 8, 14], [3, 3.5, 1.7, 10, 17, 8], [17, 8, 0.05, 10, 0.1, 14]]),
                   "c": np.array([1, 1.2, 3, 3.2]),
                   "p": np.array([[0.1312, 0.1696, 0.5569, 0.0124, 0.8283, 0.5886], [0.2329, 0.4135, 0.8307, 0.3736, 0.1004, 0.9991], [0.2348, 0.1415, 0.3522, 0.2883, 0.3047, 0.6650], [0.4047, 0.8828, 0.8732, 0.5743, 0.1091, 0.0381]])},
            "f5": {"a": np.array([[4, 4, 4, 4], [1, 1, 1, 1], [8, 8, 8, 8], [6, 6, 6, 6], [3, 7, 3, 7], [2, 9, 2, 9], [5, 5, 3, 3], [8, 1, 8, 1], [6, 2, 6, 2], [7, 3.6, 7, 3.6]]),
                   "c": np.array([0.1, 0.2, 0.2, 0.4, 0.4, 0.6, 0.3, 0.7, 0.5, 0.5])}
        }
        benchmarks = {
            "f1": lambda z: 4*z[0]**2 - 2.1*z[0]**4 + (z[0]**6)/3 + z[0]*z[1] - 4*z[1]**2 + 4*z[1]**4,
            "f2": lambda z: (z[1] - (5.1/(4*np.pi**2))*z[0]**2 + (5/np.pi)*z[0] -6)**2 + 10*(1 - (1/(8*np.pi)))*np.cos(z[0]) + 10,
            "f3": lambda z: -np.sum(benchmarks_coeffs["f3"]["c"] * np.exp(-np.sum(list(map(lambda ai, pi: ai*(z - pi)**2, benchmarks_coeffs["f3"]["a"], benchmarks_coeffs["f3"]["p"])), axis=1))),
            "f4": lambda z: -np.sum(benchmarks_coeffs["f4"]["c"] * np.exp(-np.sum(list(map(lambda ai, pi: ai*(z - pi)**2, benchmarks_coeffs["f4"]["a"], benchmarks_coeffs["f4"]["p"])), axis=1))),
            "f5": lambda z: -np.sum(list(map(lambda ai, ci: 1/((z - ai) @ (z - ai).T + ci), benchmarks_coeffs["f5"]["a"], benchmarks_coeffs["f5"]["c"])))
        }
        func_value = benchmarks.get(self.benchmark_selected)(x)
        return func_value
    
    def initRandomSoln(self):
        self.x = np.random.rand(self.n_x) * (self.ub - self.lb) + self.lb
        assert(self.isFeasible(self.x))
        self.fvalue = self.getFValue(self.x)
    
    def getNeighbourSoln(self):
        r = np.random.rand(self.n_x)
        x_new = self.x + r * (self.ub - self.x) + (1 - r) * (self.lb - self.x)
        assert(self.isFeasible(x_new))
        return x_new

In [61]:
prob = Function()
prob.setBenchmarkFunction("f3")
prob.getFValue(np.array([0, 1, 0]))

-0.00027353676805606943